In [22]:
# Purpose:  Model topics for group 6 ( 10th republican debate )
# Author:  Preeti Javaji
# Date:  05/17/16
# Course:  MAS DSE capstone, Spring 2016

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import math

pd.set_option('display.max_colwidth', 240)
pd.set_option('display.max_columns', 50)

import sys
import os
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB


print 'done'

done


In [24]:
# Function defs.

# return string with non-ascii chars removed
def to_ascii(s):
    returnstr = s.strip()
    returnstr = "".join([ch for ch in returnstr if ord(ch)< 128])
    return returnstr

print 'done'

done


In [25]:
# Load data file & clean the 'text' column.

df = pd.read_csv("Sentiment_backup.csv")

# Convert to ascii
df['ascii'] = df['text'].apply(to_ascii)
#df['ascii'] = df['ascii'].str.encode('ascii',errors='ignore') 
df['ascii'] = df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
df['ascii'] = df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
df['ascii'] = df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
df['ascii'] = df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
df['ascii'] = df['ascii'].str.lower()

df['K_sentiment'] = df['sentiment']
print df.shape
#print df[df['ascii']!=df['text']][['ascii', 'K_sentiment']].head(10)
df[['text', 'ascii']].head(3)
#df[df['ascii'].str.contains("\n")][['text', 'ascii']].head(10)

(4333, 23)


,text,ascii
0,RT @TheBaxterBean: Scott Walker's Abortion Ban Allows Rapist Father To Sue For Emotional Distress http://t.co/rHMvgumuir #GOPDebate @Badger��_,rt @thebaxterbean scott walkers abortion ban allows rapist father to sue for emotional distress url #gopdebate @badger_
1,"RT @mydaughtersarmy: Dear GOP,\rIf you want to stop abortions... \r\r#GOPDebate \rhttp://t.co/RbL1t4iPx6","rt @mydaughtersarmy dear gop,\rif you want to stop abortions... \r\r#gopdebate \rurl"
2,RT @Bipartisanism: A message to the #GOPDebate candidates: http://t.co/WFK5CmVgiN,rt @bipartisanism a message to the #gopdebate candidates url


In [26]:
# Show the class frequencies.
counts = df['subject_matter'].value_counts()
total = counts.sum()

counts = counts.apply(lambda x: ("Count: %d" % x, "Frequency: %.2f" % (float(x)/total)) )

#print("%.2f" % a)
#string = 'string%d' % (i,)

print counts
print type(counts)

None of the above                       (Count: 1271, Frequency: 0.33)
Religion                                 (Count: 407, Frequency: 0.11)
Women's Issues (not abortion though)     (Count: 362, Frequency: 0.10)
Racial issues                            (Count: 353, Frequency: 0.09)
Abortion                                 (Count: 293, Frequency: 0.08)
Jobs and Economy                         (Count: 251, Frequency: 0.07)
Immigration                              (Count: 211, Frequency: 0.06)
FOX News or Moderators                   (Count: 188, Frequency: 0.05)
LGBT issues                              (Count: 126, Frequency: 0.03)
iran deal                                 (Count: 74, Frequency: 0.02)
Healthcare (including Medicare)           (Count: 67, Frequency: 0.02)
Gun Control                               (Count: 61, Frequency: 0.02)
Foreign Policy                            (Count: 41, Frequency: 0.01)
ISIS                                      (Count: 40, Frequency: 0.01)
none  

In [27]:
%pwd

u'/Users/preetijavaji/Documents/DSE/capstone'

In [35]:
# Add the 1st democratic debate transcript
D1_df = pd.read_csv("labeled_transcripts/R10_proc_tagged.csv")
D1_df.head()

,Unnamed: 0,duration,speaker,speaker_type,start_time,text,Topic
0,0,50,APPLAUSE,audience,99,(applause),none
1,1,50,APPLAUSE,audience,883,[applause],none
2,2,50,APPLAUSE,audience,1304,[applause],none
3,3,50,APPLAUSE,audience,1331,[applause],none
4,4,50,APPLAUSE,audience,1383,[applause],none


In [37]:
#clean it up too
D1_df = D1_df[~D1_df['Topic'].isin(['none'])]
D1_df['ascii'] = D1_df['text'].apply(to_ascii)

D1_df['ascii'] = D1_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D1_df['ascii'] = D1_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D1_df['ascii'] = D1_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D1_df['ascii'] = D1_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D1_df['ascii'] = D1_df['ascii'].str.lower()
D1_df.head()

,Unnamed: 0,duration,speaker,speaker_type,start_time,text,Topic,ascii
9,9,2904,BLITZER,moderator,0,BLITZER: We're live here at the University of Houston for the 10th Republican presidential debate. An enthusiastic crowd is on hand here in the beautiful opera house at the Moore School of Music. Texas is the biggest prize next Tuesday...,intro,"blitzer were live here at the university of houston for the 10th republican presidential debate. an enthusiastic crowd is on hand here in the beautiful opera house at the moore school of music. texas is the biggest prize next tuesday, ..."
11,11,466,CARSON,candidate,2904,"CARSON: If someone had tried to describe today's America to you 30 years ago, you would have listened in disbelief. Americans know that our nation is heading off the abyss of destruction, secondary to divisiveness, fiscal irresponsibili...",leadership,"carson if someone had tried to describe todays america to you 30 years ago, you would have listened in disbelief. americans know that our nation is heading off the abyss of destruction, secondary to divisiveness, fiscal irresponsibility..."
14,14,753,KASICH,candidate,3395,"KASICH: Well, you know, on the way over here, even getting ready earlier and sitting in the green room and watching the early coverage, you know, my father carried mail on his back and his father was a coal miner and my mother's mother ...",character,"kasich well, you know, on the way over here, even getting ready earlier and sitting in the green room and watching the early coverage, you know, my father carried mail on his back and his father was a coal miner and my mothers mother wa..."
17,17,808,RUBIO,candidate,4171,"RUBIO: Well, thank you. This election, we have to decide the identity of America in the 2ist century, but as part of this primary, we have to find out our identity as a party and as a movement. Thirty-six years ago, Ronald Reagan and Ge...",gop,"rubio well, thank you. this election, we have to decide the identity of america in the 2ist century, but as part of this primary, we have to find out our identity as a party and as a movement. thirty-six years ago, ronald reagan and geo..."
21,21,610,CRUZ,candidate,5001,"CRUZ: Welcome to Texas. Here, Texas provided my family with hope. Here, my mom became the first in her family ever to go to college. Here, my dad fled Cuba and washed dishes, making 50 cents an hour to pay his way through the Universit...",character,"cruz welcome to texas. here, texas provided my family with hope. here, my mom became the first in her family ever to go to college. here, my dad fled cuba and washed dishes, making 50 cents an hour to pay his way through the university..."


In [38]:
# Now take the text and topic from both the dataframes and concat
D1_df = D1_df.rename(columns = {'Topic':'topic'})
D1_new = D1_df[['ascii','topic']]
T_DF = df[['ascii','subject_matter']]

T_DF = T_DF.rename(columns = {'subject_matter':'topic'})

In [41]:
# Now read hashtag tweets
D2_df = pd.read_csv("labeled_transcripts/HashtagTweets_updated_1.csv")
D2_df.head()

,topic,tweetid,text
0,benghazi,7.050000e+17,"""We will never forget #Benghazi @HillaryClinton https://t.co/lkfUGTaca4 #veterans #honor #courage #commitment #SuperTuesday #VoteTrump """
1,benghazi,7.050000e+17,"""##Benghazi Heroes Endorse Donald Trump https://t.co/v6jUWZdy0m """
2,benghazi,7.050000e+17,""".@hillaryclinton Great shot! Was this before or after the imaginary snipers? #VinceFoster #Benghazi #Whitewater https://t.co/0WnW2z4GAm """
3,benghazi,7.050000e+17,"""(IBD) Lying About #Benghazi Terrorist Attack Sole Hillary Accomplishment - https://t.co/ncx9a5LEmk #PJNET https://t.co/3Z7RV3ydv5 123 """
4,benghazi,7.050000e+17,"""what about the parents of #benghazi victims you lied to? @HillaryClinton @sitflyr """


In [42]:
#clean it up
#clean it up too
D2_df['ascii'] = D2_df['text'].apply(to_ascii)

D2_df['ascii'] = D2_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D2_df['ascii'] = D2_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D2_df['ascii'] = D2_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D2_df['ascii'] = D2_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D2_df['ascii'] = D2_df['ascii'].str.lower()

In [43]:
D2_new = D2_df[['ascii','topic']]
T_new1 = pd.concat([T_DF,D1_new])
print T_new1.shape
T_new = pd.concat([T_new1,D2_new])
print T_new.shape

(4637, 2)
(6188, 2)


In [44]:
# clean the topics before encoding
# first convert to lower case
T_new['topic'] = T_new['topic'].str.lower()
a = T_new['topic'].unique()
topics = list(a)
topics.sort()
topics

[nan,
 'abortion',
 'american leadership',
 'apple court order',
 'arab spring',
 'asia-pacific defense',
 'benghazi',
 'border',
 'border security',
 'bordersecurity',
 'bush impeachment',
 'campaign finance',
 'candidate tax returns',
 'change',
 'character',
 'clinton emails',
 'cnn',
 'cnndebate',
 'cnntownhall',
 'college',
 'common core',
 'conservatism',
 'corruption',
 'corruption, wall street',
 'deathpenalty',
 'deficit',
 'economy',
 'environment',
 'experience',
 'family leave',
 'familyleave',
 'flintwater',
 'flintwatercrisis',
 'foreign aid',
 'foreign policy',
 'fox news or moderators',
 'gop',
 'gun control',
 'healing hands movie',
 'healthcare',
 'healthcare (including medicare)',
 'hillary',
 'hispanic vote',
 'homelandsecurity',
 'immigration',
 'integrity',
 'intro',
 'iran',
 'iran deal',
 'iraq',
 'irs',
 'isis',
 'israel',
 'jobs',
 'jobs and economy',
 'leadership',
 'lgbt issues',
 'libya',
 'marijuana',
 'national security',
 'none',
 'none of the above',
 '

# After seeing the list above we will combine:
1.'abortion', 'planned parenthood',
2.'cnn','cnndebate','cnntownhall',
3.'corruption','corruption, wall street',
4.'family leave','familyleave',
5.'flintwater','flintwatercrisis', 
6.'foreign policy','asia-pacific defense','foreign aid', 'homelandsecurity','libya','national security','north korea','strong military', 'ukraine',
7.'healthcare','healthcare (including medicare)', 'obamacare',
8.'immigration', 'border','border security','bordersecurity', 'hispanic vote','wall',
9.'iran','iran deal',
10.'isis','radical islam',
11.'jobs','jobs and economy','deficit','economy', 'taxes','irs','trade','trade deficit',
12.'none','none of the above',
13.'religion','religious liberty',
14.'rubio',"rubio's credit cards",
15.'supreme court','supremecourt',
16.'trump','trump funding candidates','trump taxes','trump university','trump wealth',
17.'veterans','vets',
18.'zika','zikavirus'



In [47]:
T_new.ix[T_new['topic'] == 'planned parenthood','topic'] = 'abortion'  #1
T_new.ix[T_new['topic'].isin( ['cnndebate','cnntownhall']),'topic'] = 'cnn' #2
T_new.ix[T_new['topic'] == 'corruption, wall street'] = 'corruption' #3
T_new.ix[T_new['topic'] == 'familyleave','topic'] = 'family leave'  #4
T_new.ix[T_new['topic'] == 'flintwatercris','topic'] = 'flintwater' #5
T_new.ix[T_new['topic'].isin( ['asia-pacific defense','foreign aid', 'homelandsecurity','libya',
             'national security','north korea','strong military', 'ukraine']),'topic'] = 'foreign policy' #6
T_new.ix[T_new['topic'].isin( ['healthcare (including medicare)', 'obamacare']),'topic'] = 'healthcare' #7
T_new.ix[T_new['topic'].isin( ['border','border security','bordersecurity', 'hispanic vote',
                               'wall']),'topic'] = 'immigration' #8
T_new.ix[T_new['topic'] == 'iran deal','topic'] = 'iran' #9
T_new.ix[T_new['topic'] == 'radical islam','topic'] = 'isis' #10
T_new.ix[T_new['topic'].isin( ['jobs','deficit','economy', 'taxes','irs',
                               'trade','trade deficit']),'topic']= 'jobs and economy' #11
T_new.ix[T_new['topic'] == 'none of the above','topic'] = 'none' #12
T_new.ix[T_new['topic'] == 'religious liberty','topic'] = 'religion' #13
T_new.ix[T_new['topic'] == "rubio's credit cards",'topic'] = 'rubio' #14
T_new.ix[T_new['topic'] == 'supremecourt','topic'] = 'supreme court' #15
T_new.ix[T_new['topic'].isin( ['trump funding candidates','trump taxes','trump university',
                               'trump wealth']),'topic'] = 'trump'   #16
T_new.ix[T_new['topic'] == 'vets','topic'] = 'veterans' #17
T_new.ix[T_new['topic'] == 'zika','topic'] = 'zikavirus' #18




In [48]:
a = T_new['topic'].unique()
topics = list(a)
topics.sort()
topics

[nan,
 'abortion',
 'american leadership',
 'apple court order',
 'arab spring',
 'benghazi',
 'bush impeachment',
 'campaign finance',
 'candidate tax returns',
 'change',
 'character',
 'clinton emails',
 'cnn',
 'college',
 'common core',
 'conservatism',
 'corruption',
 'deathpenalty',
 'environment',
 'experience',
 'family leave',
 'flintwater',
 'flintwatercrisis',
 'foreign policy',
 'fox news or moderators',
 'gop',
 'gun control',
 'healing hands movie',
 'healthcare',
 'hillary',
 'immigration',
 'integrity',
 'intro',
 'iran',
 'iraq',
 'isis',
 'israel',
 'jobs and economy',
 'leadership',
 'lgbt issues',
 'marijuana',
 'none',
 'obama',
 'partisan politics',
 'peace',
 'politicians',
 'poverty',
 'protectionism',
 'puerto rico',
 'racial issues',
 'religion',
 'ronaldreagan',
 'rubio',
 'social security',
 'socialism',
 'supreme court',
 'syria',
 'torture',
 'trump',
 'veterans',
 'washington',
 "women's issues (not abortion though)",
 'zikavirus']

In [49]:
# remove intro closing nan and flipflopping 

T_new =T_new[~T_new['topic'].isin(['intro','flipflopping','closing','hillary','trump','rubio'])]
T_new.shape

(6143, 2)

In [50]:
# set all nans in topic to none
T_new.ix[T_new['topic'].isnull(),'topic'] = 'none'

In [51]:
T_new['topic'].value_counts()

none                                    1833
religion                                 416
women's issues (not abortion though)     362
racial issues                            353
abortion                                 294
jobs and economy                         280
immigration                              254
cnn                                      228
fox news or moderators                   188
healthcare                               188
socialism                                137
iran                                     134
lgbt issues                              126
israel                                   104
gun control                              103
isis                                      98
flintwatercrisis                          95
syria                                     86
benghazi                                  85
veterans                                  74
poverty                                   72
supreme court                             68
foreign po

In [52]:
# Sample the data so as to get equal numbers of each class.
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(T_new['ascii'],T_new['topic'], test_size=0.3, random_state=42)


In [53]:
    
print len(X_train), len(X_test) , len(y_train) , len(y_test)

4300 1843 4300 1843


In [54]:
stop_words = pd.read_table('/Users/preetijavaji/Documents/DSE/capstone/stop_words.org',names='words')
stop_words_list = list(stop_words['w'])
from sklearn.feature_extraction.text import CountVectorizer
vectorizer   =  CountVectorizer(min_df = 1,stop_words = stop_words_list, ngram_range = (1,2))
X_f_train = vectorizer.fit_transform(X_train)

In [55]:
classifiers = []
#classifiers.append( svm.SVC() )
classifiers.append( OneVsOneClassifier(LinearSVC(random_state=0)) )
classifiers.append( OneVsRestClassifier(LinearSVC(random_state=0)) )
classifiers.append( OneVsRestClassifier(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)) )

#classifier_1 = OneVsOneClassifier(svm.SVC())
#OneVsOneClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X)

cv = 10  # Number of crossvalidation folds to use.
scores = []

for classifier in classifiers:
    scores.append( cross_validation.cross_val_score(classifier, X_f_train, y_train, cv=cv) )
    
for score in scores:
    print score
    print score.mean()
    print

print 'done'

[ 0.7527115   0.77181208  0.78231293  0.76765376  0.79723502  0.77230047
  0.77857143  0.76144578  0.80825243  0.79753086]
0.778982625661

[ 0.76355748  0.80089485  0.79365079  0.7881549   0.8202765   0.77464789
  0.79761905  0.79759036  0.81796117  0.8       ]
0.79543529886

[ 0.71800434  0.71364653  0.75056689  0.72209567  0.75115207  0.70892019
  0.75714286  0.75180723  0.75970874  0.74320988]
0.737625439823

done


/Users/preetijavaji/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


In [56]:
XT_f = vectorizer.transform(X_test)
predicteds = []
for classifier in classifiers:
    classifier.fit(X_f_train, y_train)
    yy = classifier.predict(XT_f)
    print yy.shape
    predicteds.append( classifier.predict(XT_f))


(1843,)
(1843,)
(1843,)


In [57]:
# Apply classifiers to the whole (not sampled) dataset and look at the confusion matrix and classification report for each.

# Format the whole dataset.
X_all = T_new['ascii'].values
y_all = T_new['topic'].values


# Create feature vectors from the tweets. Use the same feature vector as above.
X_featurized_all = vectorizer.transform(X_all)

# Generate predictions using the classifier trained on the equally sampled data.
predicteds = []
for classifier in classifiers:
    predicteds.append( classifier.predict(X_featurized_all) )
    
for y_pred_all in predicteds:
    print accuracy_score(y_all, y_pred_all)
    #print '   Neg ', 'Neu ', 'Pos'
    print confusion_matrix(y_all, y_pred_all)
    print classification_report(y_all, y_pred_all)
    print


0.921699495361
[[249   0   0 ...,   0  11   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0  12 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,   1   0   0]
 [  4   0   0 ...,   0 321   0]
 [  0   0   0 ...,   0   0  28]]
             precision    recall  f1-score   support

   abortion       0.95      0.85      0.90       294
american leadership       0.00      0.00      0.00         1
apple court order       1.00      0.92      0.96        13
arab spring       1.00      1.00      1.00         1
   benghazi       0.97      1.00      0.98        85
bush impeachment       1.00      1.00      1.00         2
campaign finance       1.00      1.00      1.00         1
candidate tax returns       1.00      0.40      0.57         5
     change       1.00      1.00      1.00         1
  character       1.00      1.00      1.00         5
clinton emails       1.00      0.89      0.94        35
        cnn       0.96      0.97      0.97       228
    college       1.00      0.89      0.94        19
commo

/Users/preetijavaji/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [58]:
Tweet_df = pd.read_table('Tweet_g5.csv',sep=',',names=['tweetid','tweet','c3','c4','c5','c6','c7','c8'])
Tweet_df

,tweetid,tweet,c3,c4,c5,c6,c7,c8
0,NaN,"""We NEED every vote from #NEVADA for #TRUMP It's time 4 American Citizens puppets will not stop refugees or illegal https://t.co/c5tQpZmVnl """,Tweet_2016_2_23,701786301851426817,1456153867000,NaN,Mon Feb 22 15:11:07 +0000 2016,18
1,7.017863e+17,"""RT @TrussElise: We NEED every vote from #NEVADA for #TRUMP It's time 4 American Citizens puppets will not stop refugees or illegal https://… """,Tweet_2016_2_23,701919397326225410,1456185600011,NaN,Tue Feb 23 00:00:00 +0000 2016,0
2,NaN,"""Elohim In Jesus Name I pray 4 protection Psalm27 Victory #VoteTrumpNV #DonaldTrump #NVCaucus#Trump2016 #TrumpTrain https://t.co/UhyJYhVofG """,Tweet_2016_2_23,701919276597284864,1456185571000,NaN,Mon Feb 22 23:59:31 +0000 2016,0
3,7.019193e+17,"""RT @PastorOmayra: Elohim In Jesus Name I pray 4 protection Psalm27 Victory #VoteTrumpNV #DonaldTrump #NVCaucus#Trump2016 #TrumpTrain https:… """,Tweet_2016_2_23,701919397284356096,1456185600001,NaN,Tue Feb 23 00:00:00 +0000 2016,0
4,NaN,"""She also backed NAFTA &amp; Colombia/Korea FTA - latter she lobbied for after pledging to oppose https://t.co/2TAwC3aSNT https://t.co/Rv91h16ape """,Tweet_2016_2_23,701874326195171328,1456174854000,NaN,Mon Feb 22 21:00:54 +0000 2016,81
5,7.018743e+17,"""RT @davidsirota: She also backed NAFTA &amp; Colombia/Korea FTA - latter she lobbied for after pledging to oppose https://t.co/2TAwC3aSNT https… """,Tweet_2016_2_23,701919397351399424,1456185600017,NaN,Tue Feb 23 00:00:00 +0000 2016,0
6,NaN,"""Somebody tell @BernieSanders these are nouns https://t.co/0fyVWtgebc """,Tweet_2016_2_23,701866937794502656,1456173092000,NaN,Mon Feb 22 20:31:32 +0000 2016,473
7,7.018669e+17,"""RT @artist4ever: Somebody tell @BernieSanders these are nouns https://t.co/0fyVWtgebc """,Tweet_2016_2_23,701919397452103680,1456185600041,NaN,Tue Feb 23 00:00:00 +0000 2016,0
8,NaN,"""Donald Trump’s remarks on pigs’ blood elicit challenge from sister of Chapel Hill victim https://t.co/2YbcDVtGIm """,Tweet_2016_2_23,701918404639379456,1456185363000,NaN,Mon Feb 22 23:56:03 +0000 2016,24
9,7.019184e+17,"""RT @nytimes: Donald Trump’s remarks on pigs’ blood elicit challenge from sister of Chapel Hill victim https://t.co/2YbcDVtGIm """,Tweet_2016_2_23,701919398068510720,1456185600188,NaN,Tue Feb 23 00:00:00 +0000 2016,0


In [59]:
# copy classifier 0 as the model for g1 and test it in spark on a sample 
# Convert to ascii
Tweet_df['ascii'] = Tweet_df['tweet'].apply(to_ascii)

# Remove hashtags
#df['ascii'] = df['ascii'].str.replace(r"#([A-Za-z0-9_]+)", " ")

# Remove handles
#df['ascii'] = df['ascii'].str.replace(r"@([A-Za-z0-9_]+)", " ")

# Remove URLs
#df['ascii'] = df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", " ")
# Replace URLs with "URL"
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")
#df['ascii'] = df['ascii'].str.replace(r"(['';:@%#()\+\*\"\…\“\”])", "")
#df['ascii'] = df['ascii'].str.replace(r"([;:@%#()\+\*\"\…\“\”])", "")

# Remove eol symbols
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbolst
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
Tweet_df['ascii'] = Tweet_df['ascii'].str.lower()


Tweet_df[['tweet', 'ascii']].head(3)
#df[df['ascii'].str.contains("\n")][['text', 'ascii']].head(10)


,tweet,ascii
0,"""We NEED every vote from #NEVADA for #TRUMP It's time 4 American Citizens puppets will not stop refugees or illegal https://t.co/c5tQpZmVnl """,we need every vote from #nevada for #trump its time 4 american citizens puppets will not stop refugees or illegal url
1,"""RT @TrussElise: We NEED every vote from #NEVADA for #TRUMP It's time 4 American Citizens puppets will not stop refugees or illegal https://… """,rt @trusselise we need every vote from #nevada for #trump its time 4 american citizens puppets will not stop refugees or illegal url
2,"""Elohim In Jesus Name I pray 4 protection Psalm27 Victory #VoteTrumpNV #DonaldTrump #NVCaucus#Trump2016 #TrumpTrain https://t.co/UhyJYhVofG """,elohim in jesus name i pray 4 protection psalm27 victory #votetrumpnv #donaldtrump #nvcaucus#trump2016 #trumptrain url


In [60]:
X = Tweet_df['ascii'].values

#train_vectors = vectorizer.fit_transform(train_data)
#test_vectors = vectorizer.transform(test_data)
X_featurized = vectorizer.transform(X)
        
print X_featurized.shape
print 'done'

(10000, 35351)
done


In [61]:
predicted_new = []
for classifier in classifiers:
    predicted_new.append( classifier.predict(X_featurized) )

In [68]:
Tweet_df['topic'] = predicted_new[2]

In [69]:
from collections import Counter

Keys = Counter(predicted_new[2]).keys() # equals to list(set(words))
values =  Counter(predicted_new[2]).values() # counts the elements' frequency
for k,v in zip(Keys,values):
    print k,':',v

racial issues : 133
fox news or moderators : 6
supreme court : 327
deathpenalty : 1
lgbt issues : 12
environment : 2
religion : 194
clinton emails : 7
cnn : 458
veterans : 14
benghazi : 22
israel : 20
iran : 17
immigration : 239
marijuana : 7
poverty : 32
iraq : 4
obama : 4
none : 7820
isis : 14
flintwatercrisis : 11
gun control : 45
jobs and economy : 49
abortion : 68
zikavirus : 1
socialism : 263
conservatism : 2
healthcare : 74
syria : 13
women's issues (not abortion though) : 141


In [71]:
Tweet_df[Tweet_df['topic'] == "flintwatercrisis"]['tweet']

208                    "Elijah Cummings says Republicans are soft on Rick Snyder over Flint water crisis https://t.co/KaALt81KBP https://t.co/VIgxpQV2tq "
209        "RT @HuffPostPol: Elijah Cummings says Republicans are soft on Rick Snyder over Flint water crisis https://t.co/KaALt81KBP https://t.co/VIgx… "
2824    "blaha_b: RT DinkLane: #JoeScarborough &amp; #MikeBrzezinski in trouble 4 #Trump Support--Turn 2 #FlintWaterCrisis MISS… https://t.co/7SrPu79tbm "
2929       "The Flint disaster is Rick Snyder’s fault https://t.co/nSM7pe4ui5 Rick Snyder is only the tip of the iceberg, all gop guvs are Koch viceroys "
2930          "#ArrestGovSnyder HE appointed a succession of Emergency Managers who made catastrophic decisions to save a few $$ https://t.co/IFRwVylCra "
3369                                 "The #PoliticalRevolution is now #NotMeUs   #WeAreBernie #BernieSanders #SCPrimary #Mapoli  https://t.co/LS3PVxlnO7 "
3717                                                             "Angr

In [73]:
import pickle
# save model in a pickle file
output = open('mymodel_g5.pkl', 'wb')
pickle.dump(classifiers[2],output)
output.close()

In [75]:
vec_file = open('myvec_g5.pkl','wb')
pickle.dump(vectorizer,vec_file)
vec_file.close()